# Preparing Data and a Linear Model

**Learning Objectives**
1. Explore the data with some EDA
2. Find correlations
3. Create moving average and RSI features
4. Create features and targets
5. Check the correlations
6. Create train and test features
7. Fit a linear model
8. Evaluate our results


In this notebook we will also explore some stock data, and prepare it for machine learning algorithms. Finally, we will fit our first machine learning model – a linear model, in order to predict future price changes of stocks.

## Explore the data with some Exploratory Data Analysis (EDA)
First, let's explore the data. Any time we begin a machine learning (ML) project, we need to first do some exploratory data analysis (EDA) to familiarize ourselves with the data. This includes things like:

* raw data plots
* histograms
* and more...

Let's begin with raw data plots and histograms. This allows us to understand our data's distributions. If it's a normal distribution, we can use things like parametric statistics.

First, let's load two stocks into pandas DataFrames (LNG and SPY):

In [ ]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install TA-Lib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install statsmodels

import pandas as pd
import matplotlib.pyplot as plt
import talib
import seaborn as sns
import statsmodels.api as sm
import numpy as np

lng_df = pd.read_csv('data/lng.csv')
spy_df = pd.read_csv('data/spy.csv')

There are now two stocks loaded for us into pandas DataFrames: `lng_df` and `spy_df` (LNG and SPY). Take a look at them with `.head()`. We'll use the closing prices and eventually volume as inputs to ML algorithms.

Note: We'll call `plt.clf()` each time we want to make a new plot, or `f = plt.figure()`.

### Instructions
* Print out the first 5 lines of the two DataFrame (`lng_df` and `spy_df`) and examine their contents.
* Use the pandas library to plot raw time series data for `'SPY'` and `'LNG'` with the adjusted close price (`'Adj_Close'`) – set `legend = True` in `.plot()`.
* Use `plt.show()` to show the raw time series plot (we imported `matplotlib.pyplot` as `plt`).
* Use the pandas and matplotlib to make a histogram of the adjusted close 1-day percent difference (use `.pct_change()`) for SPY and LNG.

In [ ]:
print(lng_df.head())  # examine the DataFrames
print(____)  # examine the SPY DataFrame

# Plot the Adj_Close columns for SPY and LNG
spy_df[____].plot(label='SPY', legend=True)
lng_df[____].plot(label=____, ____, secondary_y=True)
____  # show the plot
plt.clf()  # clear the plot space

# Histogram of the daily price change percent of Adj_Close for LNG
lng_df['Adj_Close'].____.plot.hist(bins=50)
plt.xlabel('adjusted close 1-day percent change')
plt.show()

Good work, now we know what our data looks like!

## Correlations
Correlations are nice to check out before building machine learning models, because we can see which features correlate to the target most strongly. Pearson's correlation coefficient is often used, which only detects linear relationships. It's commonly assumed our data is normally distributed, which we can "eyeball" from histograms. Highly correlated variables have a Pearson correlation coefficient near 1 (positively correlated) or -1 (negatively correlated). A value near 0 means the two variables are not linearly correlated.

If we use the same time periods for previous price changes and future price changes, we can see if the stock price is mean-reverting (bounces around) or trend-following (goes up if it has been going up recently).

### Instructions
Using the `lng_df` DataFrame and its `Adj_Close`:
* Create the 5-day future price (as `5d_future_close`) with pandas' `.shift(-5).
* Use `pct_change(5)` on `5d_future_close` and `Adj_Close` to create the future 5-day % price change (`5d_close_future_pct`), and the current 5-day % price change (`5d_close_pct`).
* Examine correlations between the two 5-day percent price change columns with `.corr()` on `lng_df`.
* Using `plt.scatter()`, make a scatterplot of `5d_close_pct` vs `5d_close_future_pct`.

In [ ]:
# Create 5-day % changes of Adj_Close for the current day, and 5 days in the future
lng_df['5d_future_close'] = lng_df['Adj_Close'].shift(____)
lng_df['5d_close_future_pct'] = lng_df['5d_future_close'].pct_change(5)
lng_df['5d_close_pct'] = lng_df['Adj_Close'].pct_change(____)

# Calculate the correlation matrix between the 5d close pecentage changes (current and future)
corr = lng_df[['5d_close_pct', '5d_close_future_pct']].____
print(corr)

# Scatter the current 5-day percent change vs the future 5-day percent change
plt.scatter(lng_df['5d_close_pct'], lng_df[____])
plt.show()

Great work! We can see the 5-day change is slightly negatively correlated to the change in the last 5 days – an example of overall mean reversion!

## Create moving average and RSI features

We want to add historical data to our machine learning models to make better predictions, but adding lots of historical time steps is tricky. Instead, we can condense information from previous points into a single timestep with indicators.

A moving average is one of the simplest indicators - it's the average of previous data points. This is the function `talib.SMA()` from the `TAlib` library.

Another common technical indicator is the relative strength index (RSI). The *n* periods is set in `talib.RSI()` as the `timeperiod` argument.

A common period for RSI is 14, so we'll use that as one setting in our calculations.

### Instructions
* Create a list of feature names (start with a list containing only `'5d_close_pct'`).
* Use timeperiods of 14, 30, 50, and 200 to calculate moving averages with `talib.SMA()` from adjusted close prices (`lng_df['Adj_Close']`).
* Normalize the moving averages with the adjusted clsoe by dividing by `Adj_Close`.
* Within the loop, calculate RSI with `talib.RSI()` from `Adj_Close` and using `n` for the timeperiod.

In [ ]:
feature_names = ____  # a list of the feature names for later

# Create moving averages and rsi for timeperiods of 14, 30, 50, and 200
for n in [____]:

    # Create the moving average indicator and divide by Adj_Close
    lng_df['ma' + str(n)] = talib.SMA(lng_df['Adj_Close'].values,
                              timeperiod=n) / lng_df[____]
    # Create the RSI indicator
    lng_df['rsi' + str(n)] = talib.____(lng_df['Adj_Close'].____, timeperiod=____)
    
    # Add rsi and moving average to the feature name list
    feature_names = feature_names + ['ma' + str(n), 'rsi' + str(n)]

print(feature_names)

Nice job, now we have our indicators!

## Create features and targets

We *almost* have features and targets that are machine-learning ready – we have features from current price changes (`5d_close_pct`) and indicators (moving averages and RSI), and we created targets of future price changes (`5d_close_future_pct`). Now we need to break these up into separate numpy arrays so we can feed them into machine learning algorithms.

Our indicators also cause us to have missing values at the beginning of the DataFrame due to the calculations. We could backfill this data, fill it with a single value, or drop the rows. Dropping the rows is a good choice, so our machine learning algorithms aren't confused by any sort of backfilled or 0-filled data. Pandas has a `.dropna()` function which we will use to drop any rows with missing values.

### Instructions
* Drop the missing values from `lng_df` with the `.dropna()` from pandas.
* Create a variable containing our targets which are the `'5d_close_future_pct'` values.
* Create a DataFrame containing both targets (`5d_close_future_pct`) *and* features (contained in the existing list `feature_names`) so we can check the correlations.

In [ ]:
# Drop all na values
lng_df = lng_df.____

# Create features and targets
# use feature_names for features; '5d_close_future_pct' for targets
features = lng_df[feature_names]
targets = lng_df[____]

# Create DataFrame from target column and feature columns
feature_and_target_cols = ['5d_close_future_pct'] + ____
feat_targ_df = lng_df[feature_and_target_cols]

# Calculate correlation matrix
corr = feat_targ_df.corr()
print(corr)

Nice job, now we've got features and targets ready for machine learning!

## Check the correlations

Before we fit our first machine learning model, let's look at the correlations between the features and the targets. Ideally we want large (near 1 or -1) correlations between features and targets. Examining correlations can help us tweak features to maximize correlation (for example, altering the `timeperiod` argument in the `talib` functions). It can also help us remove features that aren't correlated to the target.

To easily plot a correlation matrix, we can use `seaborn`'s `heatmap()` function. This takes a correlation matrix as the first argument, and has many other options. We will use the `annot` option to turn on annotations.

### Instructions (1/2)
* Plot a heatmap of the correlation matrix (`corr`) we calculated in the last exercise (we've already imported `seaborn` as `sns`).
* Turn on annotations using the `sns.heatmap()` option `annot=True`
* Show the plot with `plt.show()`.

In [ ]:
# Plot heatmap of correlation matrix
sns.heatmap(____, annot= ____, annot_kws = {"size": 14})
plt.yticks(rotation=0, size = 14); plt.xticks(rotation=90, size = 14)  # fix ticklabel directions and size
plt.____  # show the plot

### Instructions (2/2)
* Inspect the heatmap we've generated in the previous step. Which feature/variable exhibits the **highest correlations** with the target (`5d_close_future_pct`)?
* Clear the plot area with `plt.clf()` to prepare for our second plot.
* Create a scatter plot of the most correlated feature/variable with the target (`5d_close_future_pct`) from the `lng_df` DataFrame.


In [ ]:
# Create a scatter plot of the most highly correlated variable with the target
plt.scatter(lng_df[____], lng_df[____])
plt.show()

Nice work! We can see a few features have some correlation to the target!

In [ ]:
# Add a constant to the features
linear_features = sm.____(features)

# Create a size for the training set that is 85% of the total number of samples
train_size = int(0.85 * ____)
train_features = linear_features[:train_size]
train_targets = targets[____]
test_features = linear_features[train_size:]
test_targets = targets[train_size:]
print(linear_features.shape, train_features.shape, test_features.shape)

Good work! We're ready to fit our linear model.

## Fit a linear model

We'll now fit a linear model, because they are simple and easy to understand. Once we've fit our model, we can see which predictor variables appear to be meaningfully linearly correlated with the target, as well as their magnitude of effect on the target. Our judgment of whether or not predictors are significant is based on the p-values of coefficients. This is using a t-test to statistically test if the coefficient significantly differs from 0. The p-value is the percent chance that the coefficient for a feature does not differ from zero. Typically, we take a p-value of less than 0.05 to mean the coefficient is significantly different from 0.

### Instructions
* Fit the linear model (using the `.fit()` method) and save the results in the `results` variable.
* Print out the results summary with the `.summary()` function.
* Print out the p-values from the results (the `.pvalues` property of `results`).
* Make predictions from the `train_features` and `test_features` using the `.predict()` function of our `results` object.

In [ ]:
# Create the linear model and complete the least squares fit
model = sm.OLS(train_targets, train_features)
results = model.____  # fit the model
print(results.____)

# examine pvalues
# Features with p <= 0.05 are typically considered significantly different from 0
print(results.____)

# Make predictions from our model for train and test sets
train_predictions = results.predict(train_features)
test_predictions = ____

Nice job! Now we can evaluate the results from our predictions.

## Evaluate our results

Once we have our linear fit and predictions, we want to see how good the predictions are so we can decide if our model is any good or not. Ideally, we want to back-test any type of trading strategy. However, this is a complex and typically time-consuming experience.

A quicker way to understand the performance of our model is looking at regression evaluation metrics like R^2, and plotting the predictions versus the actual values of the targets. Perfect predictions would form a straight, diagonal line in such a plot, making it easy for us to eyeball how our predictions are doing in different regions of price changes. We can use `matplotlib`'s `.scatter()` function to create scatter plots of the predictions and actual values.

### Instructions
* Show `test_predictions` vs `test_targets` in a scatterplot, with 20% opacity for the points (use the `alpha` parameter to set opacity).
* Plot the perfect prediction line using `np.arrange()` and the minimum and maximum values for the xaxis (`xmin`, `xmax`).
* Display the legend on the plot with `plt.legend()`.

In [ ]:
# Scatter the predictions vs the targets with 20% opacity
plt.scatter(train_predictions, train_targets, alpha=0.2, color='b', label='train')
plt.scatter(____, ____, ____, color='r', label='test')

# Plot the perfect prediction line
xmin, xmax = plt.xlim()
plt.plot(np.arange(xmin, xmax, 0.01), np.arange(____, ____, 0.01), c='k')

# Set the axis labels and show the plot
plt.xlabel('predictions')
plt.ylabel('actual')
____  # show the legend
plt.show()

Good work! We can see our predictions are ok, but not very good yet. We something a bit more complex! Coming up...